# Neural Network Architecture Study - OPTIMIZED
## Paper 4 - Mit Preloading für schnellere Ausführung

**Optimierungen:**
- ✅ Daten werden in RAM vorgeladen
- ✅ Keine W&B Hook Probleme
- ✅ Schnellere Ausführung
- ✅ VS Code kompatibel

## 1. Setup & Imports

In [1]:
# Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split, TensorDataset
import matplotlib.pyplot as plt
import numpy as np
import wandb
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from tqdm import tqdm
import time
import pandas as pd

# Konfiguration
RANDOM_SEED = 42
BATCH_SIZE = 64
EPOCHS = 20
LEARNING_RATE = 0.001
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

print(f"PyTorch: {torch.__version__}")
print(f"Device: {DEVICE}")
print(f"W&B: {wandb.__version__}")

PyTorch: 2.9.1+cpu
Device: cpu
W&B: 0.23.1


## 2. W&B Login

In [ ]:
# W&B Login
wandb.login(key="API KEY HIER EINFÜGEN")
print("✅ W&B Login erfolgreich!")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Silver\_netrc
wandb: Currently logged in as: anouk347 (anouk347-university-of-applied-sciences-ansbach) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


✅ W&B Login erfolgreich!


## 3. Data Loading mit PRELOADING

Hier laden wir ALLE Daten in den RAM - viel schneller!

In [3]:
# ============================================
# DATA LOADING MIT PRELOADING
# ============================================

print("📦 Lade Fashion-MNIST in den Arbeitsspeicher...")

# Transform
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Lade Daten
train_dataset_orig = datasets.FashionMNIST('./data', train=True, download=True, transform=transform)
test_dataset_orig = datasets.FashionMNIST('./data', train=False, download=True, transform=transform)

# Klassenamen
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# PRELOAD in RAM
print("⚡ Preloading in RAM...")

# Alle Daten in Tensoren laden
train_images = []
train_labels = []
for img, label in tqdm(train_dataset_orig, desc="Loading train"):
    train_images.append(img)
    train_labels.append(label)

test_images = []
test_labels = []
for img, label in tqdm(test_dataset_orig, desc="Loading test"):
    test_images.append(img)
    test_labels.append(label)

# Zu Tensoren konvertieren
train_images = torch.stack(train_images)
train_labels = torch.tensor(train_labels)
test_images = torch.stack(test_images)
test_labels = torch.tensor(test_labels)

# Train/Val Split
train_size = int(0.8 * len(train_images))
val_size = len(train_images) - train_size

# Split
indices = torch.randperm(len(train_images), generator=torch.Generator().manual_seed(RANDOM_SEED))
train_indices = indices[:train_size]
val_indices = indices[train_size:]

# DataLoaders mit preloaded Daten
train_loader = DataLoader(
    TensorDataset(train_images[train_indices], train_labels[train_indices]),
    batch_size=BATCH_SIZE, shuffle=True
)
val_loader = DataLoader(
    TensorDataset(train_images[val_indices], train_labels[val_indices]),
    batch_size=BATCH_SIZE, shuffle=False
)
test_loader = DataLoader(
    TensorDataset(test_images, test_labels),
    batch_size=BATCH_SIZE, shuffle=False
)

print(f"✅ Daten geladen!")
print(f"Train: {len(train_indices)}, Val: {len(val_indices)}, Test: {len(test_labels)}")

📦 Lade Fashion-MNIST in den Arbeitsspeicher...
⚡ Preloading in RAM...


Loading test: 100%|██████████| 10000/10000 [00:03<00:00, 2710.41it/s]


✅ Daten geladen!
Train: 48000, Val: 12000, Test: 10000


## 4. Model Architectures

In [4]:
# ============================================
# MODEL ARCHITECTURES
# ============================================

class SimpleMLP(nn.Module):
    """Architecture A: Simple MLP"""
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(784, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )
    def forward(self, x):
        return self.layers(x)

class DeepMLP(nn.Module):
    """Architecture B: Deep MLP"""
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(784, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )
    def forward(self, x):
        return self.layers(x)

class VariableMLP(nn.Module):
    """MLP mit variabler Breite"""
    def __init__(self, hidden_size=128):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(784, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 10)
        )
    def forward(self, x):
        return self.layers(x)

class SimpleCNN(nn.Module):
    """Architecture C: Simple CNN"""
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 14 * 14, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )
    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

class DeeperCNN(nn.Module):
    """Architecture D: Deeper CNN mit BatchNorm"""
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 7 * 7, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )
    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

class CNNWithDropout(nn.Module):
    """CNN mit Dropout"""
    def __init__(self, dropout_rate=0.3):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 7 * 7, 256),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(256, 10)
        )
    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print("✅ Models definiert!")

✅ Models definiert!


## 5. Training Function (OHNE W&B Hooks - behebt das Problem!)

In [5]:
# ============================================
# TRAINING FUNCTION (Optimiert, ohne W&B Hooks)
# ============================================

def train_model(model, config, project_name="Paper_4"):
    """Trainiert Modell und loggt zu W&B - OHNE wandb.watch()"""
    
    # W&B init
    run = wandb.init(
        project=project_name,
        config=config,
        name=config.get('run_name', 'experiment'),
        reinit=True
    )
    
    model = model.to(DEVICE)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config.get('learning_rate', LEARNING_RATE))
    
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': [], 'epoch_times': []}
    best_val_acc = 0.0
    start_time = time.time()
    
    for epoch in range(config.get('epochs', EPOCHS)):
        epoch_start = time.time()
        
        # Training
        model.train()
        train_loss, train_correct, train_total = 0.0, 0, 0
        
        for images, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{config.get("epochs", EPOCHS)}'):
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = outputs.max(1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
        
        # Validation
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        # Metrics
        avg_train_loss = train_loss / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)
        train_accuracy = 100 * train_correct / train_total
        val_accuracy = 100 * val_correct / val_total
        epoch_time = time.time() - epoch_start
        
        history['train_loss'].append(avg_train_loss)
        history['train_acc'].append(train_accuracy)
        history['val_loss'].append(avg_val_loss)
        history['val_acc'].append(val_accuracy)
        history['epoch_times'].append(epoch_time)
        
        # Log to W&B
        wandb.log({
            'epoch': epoch + 1,
            'train_loss': avg_train_loss,
            'train_accuracy': train_accuracy,
            'val_loss': avg_val_loss,
            'val_accuracy': val_accuracy,
            'train_val_gap': train_accuracy - val_accuracy,
            'epoch_time': epoch_time
        })
        
        print(f'Epoch {epoch+1}/{config.get("epochs", EPOCHS)} | '
              f'Train: {train_accuracy:.2f}% | Val: {val_accuracy:.2f}% | Time: {epoch_time:.1f}s')
        
        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
    
    total_time = time.time() - start_time
    
    # Summary
    wandb.summary.update({
        'best_val_accuracy': best_val_acc,
        'final_val_accuracy': val_accuracy,
        'total_time': total_time,
        'parameters': count_parameters(model)
    })
    
    print(f'✅ Training Complete! Best Val Acc: {best_val_acc:.2f}%')
    wandb.finish()
    
    return history, model

print("✅ Training function ready!")

✅ Training function ready!


## 6. Experiment 1: MLP Depth & Width Study

In [6]:
# Experiment 1.1: Simple MLP
print("🔬 Experiment 1.1: Simple MLP")
config = {'run_name': 'exp1.1-simple-mlp', 'architecture': 'Simple MLP', 
          'epochs': 20, 'learning_rate': 0.001}
history_simple, model_simple = train_model(SimpleMLP(), config)

🔬 Experiment 1.1: Simple MLP


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


Epoch 1/20: 100%|██████████| 750/750 [00:04<00:00, 151.86it/s]


Epoch 1/20 | Train: 81.30% | Val: 84.14% | Time: 6.0s


Epoch 2/20: 100%|██████████| 750/750 [00:07<00:00, 105.46it/s]


...

Epoch 19/20: 100%|██████████| 750/750 [00:03<00:00, 248.06it/s]


Epoch 19/20 | Train: 93.34% | Val: 88.77% | Time: 3.4s


Epoch 20/20: 100%|██████████| 750/750 [00:03<00:00, 239.11it/s]


Epoch 20/20 | Train: 93.40% | Val: 88.68% | Time: 3.6s
✅ Training Complete! Best Val Acc: 88.89%


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
epoch_time,▆█▃▃▃▂▄▂▁▂▁▁▃▂▂▅▂▃▁▁
train_accuracy,▁▄▄▅▅▅▆▆▆▇▇▇▇▇▇▇████
train_loss,█▅▅▄▄▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁
train_val_gap,▁▃▄▄▄▅▅▆▅▆▆▆▇▇▇▇▇███
val_accuracy,▁▄▅▅▆▅▆▆▇▇█▇▆▇██▇▇██
val_loss,█▄▄▃▃▃▂▂▁▂▁▂▃▂▂▂▂▃▃▃
best_val_accuracy,88.89167
epoch,20
epoch_time,3.60374
final_val_accuracy,88.68333


In [7]:
# Experiment 1.2: Deep MLP
print("🔬 Experiment 1.2: Deep MLP")
config = {'run_name': 'exp1.2-deep-mlp', 'architecture': 'Deep MLP',
          'epochs': 20, 'learning_rate': 0.001}
history_deep, model_deep = train_model(DeepMLP(), config)

🔬 Experiment 1.2: Deep MLP


Epoch 1/20: 100%|██████████| 750/750 [00:05<00:00, 141.40it/s]


Epoch 1/20 | Train: 80.10% | Val: 83.07% | Time: 6.0s


Epoch 2/20: 100%|██████████| 750/750 [00:04<00:00, 150.75it/s]


Epoch 2/20 | Train: 85.84% | Val: 86.28% | Time: 5.4s


...

Epoch 19/20: 100%|██████████| 750/750 [00:04<00:00, 150.74it/s]


Epoch 19/20 | Train: 94.01% | Val: 88.92% | Time: 5.4s


Epoch 20/20: 100%|██████████| 750/750 [00:05<00:00, 135.68it/s]


Epoch 20/20 | Train: 94.12% | Val: 89.48% | Time: 5.9s
✅ Training Complete! Best Val Acc: 89.48%


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
epoch_time,▇▃▇▄▆▂▇▃▇▁▇▃▆▅█▄▆▅▃▇
train_accuracy,▁▄▅▅▅▆▆▆▆▇▇▇▇▇▇▇████
train_loss,█▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁
train_val_gap,▁▃▄▅▅▄▅▆▅▆▆▆▇▇▇█▇███
val_accuracy,▁▅▅▄▆▇▆▆▇▇▇▇▇▇▇▇██▇█
val_loss,█▄▄▄▃▂▂▂▁▁▂▂▁▂▃▃▃▃▃▄
best_val_accuracy,89.48333
epoch,20
epoch_time,5.91763
final_val_accuracy,89.48333


In [8]:
# Experiment 1.3: Width Comparison
print("🔬 Experiment 1.3: MLP Width Study")
width_experiments = [64, 128, 256, 512]
width_histories = {}

for width in width_experiments:
    print(f"\n📊 Training MLP width={width}")
    config = {'run_name': f'exp1.3-mlp-width-{width}', 
              'architecture': f'MLP Width={width}',
              'hidden_size': width, 'epochs': 20, 'learning_rate': 0.001}
    history, _ = train_model(VariableMLP(width), config)
    width_histories[width] = history

🔬 Experiment 1.3: MLP Width Study

📊 Training MLP width=64


Epoch 1/20: 100%|██████████| 750/750 [00:02<00:00, 264.93it/s]


Epoch 1/20 | Train: 81.11% | Val: 83.81% | Time: 3.2s


Epoch 2/20: 100%|██████████| 750/750 [00:03<00:00, 211.60it/s]


Epoch 2/20 | Train: 85.49% | Val: 85.51% | Time: 3.8s


...

Epoch 19/20: 100%|██████████| 750/750 [00:05<00:00, 141.53it/s]


Epoch 19/20 | Train: 94.21% | Val: 89.30% | Time: 5.9s


Epoch 20/20: 100%|██████████| 750/750 [00:05<00:00, 131.06it/s]


Epoch 20/20 | Train: 94.45% | Val: 88.74% | Time: 6.3s
✅ Training Complete! Best Val Acc: 89.30%


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
epoch_time,▇▂▂▄▂▄▃▄▁▆▃▇▂▃▁▆▅▆▆█
train_accuracy,▁▃▄▅▅▅▆▆▆▆▇▇▇▇▇▇████
train_loss,█▆▅▄▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁
train_val_gap,▁▄▄▄▅▅▅▅▅▆▆▆▇▇▇▇█▇▇█
val_accuracy,▁▂▄▅▄▆▆▇▇▆▇▇▇▆▇▇▆██▇
val_loss,█▇▄▃▄▃▂▁▁▂▁▂▃▅▃▃▆▃▄▆
best_val_accuracy,89.3
epoch,20
epoch_time,6.30119
final_val_accuracy,88.74167


## 7. Experiment 2: MLP vs CNN

In [9]:
# Experiment 2.1: Simple CNN
print("🔬 Experiment 2.1: Simple CNN")
config = {'run_name': 'exp2.1-simple-cnn', 'architecture': 'Simple CNN',
          'epochs': 20, 'learning_rate': 0.001}
history_simple_cnn, model_simple_cnn = train_model(SimpleCNN(), config)

🔬 Experiment 2.1: Simple CNN


Epoch 1/20: 100%|██████████| 750/750 [00:27<00:00, 26.97it/s]


Epoch 1/20 | Train: 84.84% | Val: 88.62% | Time: 32.1s


Epoch 2/20: 100%|██████████| 750/750 [00:29<00:00, 25.36it/s]


Epoch 2/20 | Train: 89.56% | Val: 89.72% | Time: 33.8s


...

Epoch 19/20: 100%|██████████| 750/750 [00:35<00:00, 21.28it/s]


Epoch 19/20 | Train: 99.07% | Val: 91.51% | Time: 39.3s


Epoch 20/20: 100%|██████████| 750/750 [00:31<00:00, 23.45it/s]


Epoch 20/20 | Train: 99.08% | Val: 91.25% | Time: 35.4s
✅ Training Complete! Best Val Acc: 91.51%


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
epoch_time,▁▂▂▂▂▁▂▇█▃▂▃▂▃▃▂▂▂▅▃
train_accuracy,▁▃▄▅▅▅▆▆▇▇▇▇▇███████
train_loss,█▆▅▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
train_val_gap,▁▃▃▄▅▅▅▆▆▇▇▇▇▇▇▇████
val_accuracy,▁▄▆▇▆▆▆██▇▇▆█▇███▆█▇
val_loss,▃▂▁▁▂▁▂▁▂▃▃▄▄▅▅▆▆█▇█
best_val_accuracy,91.50833
epoch,20
epoch_time,35.44862
final_val_accuracy,91.25


In [10]:
# Experiment 2.2: Deeper CNN
print("🔬 Experiment 2.2: Deeper CNN")
config = {'run_name': 'exp2.2-deeper-cnn', 'architecture': 'Deeper CNN',
          'epochs': 20, 'learning_rate': 0.001}
history_deeper_cnn, model_deeper_cnn = train_model(DeeperCNN(), config)

🔬 Experiment 2.2: Deeper CNN


Epoch 1/20: 100%|██████████| 750/750 [01:00<00:00, 12.35it/s]


Epoch 1/20 | Train: 85.69% | Val: 89.47% | Time: 67.6s


Epoch 2/20: 100%|██████████| 750/750 [01:02<00:00, 12.01it/s]


Epoch 2/20 | Train: 90.25% | Val: 90.37% | Time: 69.6s


...

Epoch 19/20: 100%|██████████| 750/750 [00:58<00:00, 12.74it/s]


Epoch 19/20 | Train: 99.06% | Val: 92.14% | Time: 65.1s


Epoch 20/20: 100%|██████████| 750/750 [00:57<00:00, 13.15it/s]


Epoch 20/20 | Train: 99.16% | Val: 91.82% | Time: 63.3s
✅ Training Complete! Best Val Acc: 92.81%


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
epoch_time,▂▂▂█▃▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁
train_accuracy,▁▃▄▅▅▅▆▆▇▇▇▇████████
train_loss,█▆▅▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
train_val_gap,▁▃▄▄▆▅▅▆▆▇▇▇▇▇▇█████
val_accuracy,▁▃▄▅▂▅▇▆▇▆▆▇█▇▇▇▇▇▇▆
val_loss,▃▂▂▁▃▂▁▁▂▃▃▄▃▅▅▅▆▇▇█
best_val_accuracy,92.80833
epoch,20
epoch_time,63.25094
final_val_accuracy,91.81667


## 8. Experiment 3: Regularization (Dropout)

In [11]:
# Experiment 3: Dropout Study
print("🔬 Experiment 3: Dropout Study")
dropout_rates = [0.0, 0.2, 0.3, 0.5]
dropout_histories = {}

for dropout in dropout_rates:
    print(f"\n📊 Training CNN with dropout={dropout}")
    config = {'run_name': f'exp3-dropout-{dropout}', 'dropout': dropout,
              'epochs': 20, 'learning_rate': 0.001}
    history, _ = train_model(CNNWithDropout(dropout), config)
    dropout_histories[dropout] = history

🔬 Experiment 3: Dropout Study

📊 Training CNN with dropout=0.0


Epoch 1/20: 100%|██████████| 750/750 [00:54<00:00, 13.81it/s]


Epoch 1/20 | Train: 85.91% | Val: 90.03% | Time: 60.4s


Epoch 2/20: 100%|██████████| 750/750 [00:59<00:00, 12.67it/s]


Epoch 2/20 | Train: 90.24% | Val: 88.38% | Time: 65.3s


...

Epoch 19/20: 100%|██████████| 750/750 [00:58<00:00, 12.78it/s]


Epoch 19/20 | Train: 96.54% | Val: 92.58% | Time: 66.1s


Epoch 20/20: 100%|██████████| 750/750 [00:57<00:00, 13.10it/s]


Epoch 20/20 | Train: 96.70% | Val: 92.31% | Time: 64.5s
✅ Training Complete! Best Val Acc: 92.72%


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
epoch_time,▁▂▁▁▂▂▁▁▁█▄▃▄▇▅▄▂▂▃▂
train_accuracy,▁▄▄▅▅▆▆▆▆▇▇▇▇▇▇█████
train_loss,█▅▅▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁
train_val_gap,▁▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇███
val_accuracy,▁▂▅▆▆▇▇▇███████▇███▇
val_loss,█▆▄▃▂▂▂▁▁▁▁▂▃▃▂▃▄▃▄▅
best_val_accuracy,92.725
epoch,20
epoch_time,64.47626
final_val_accuracy,92.30833


## 9. Experiment 4: Learning Rate Study

In [12]:
# Experiment 4: Learning Rate Study
print("🔬 Experiment 4: Learning Rate Study")
learning_rates = [0.1, 0.01, 0.001, 0.0001]
lr_histories = {}

for lr in learning_rates:
    print(f"\n📊 Training with LR={lr}")
    config = {'run_name': f'exp4-lr-{lr}', 'learning_rate': lr, 'epochs': 20}
    history, _ = train_model(DeeperCNN(), config)
    lr_histories[lr] = history

🔬 Experiment 4: Learning Rate Study

📊 Training with LR=0.1


Epoch 1/20: 100%|██████████| 750/750 [00:52<00:00, 14.19it/s]


Epoch 1/20 | Train: 69.97% | Val: 79.18% | Time: 58.2s


Epoch 2/20: 100%|██████████| 750/750 [00:56<00:00, 13.18it/s]


Epoch 2/20 | Train: 78.98% | Val: 79.45% | Time: 62.4s


...

Epoch 19/20: 100%|██████████| 750/750 [00:59<00:00, 12.70it/s]


Epoch 19/20 | Train: 97.70% | Val: 91.80% | Time: 64.9s


Epoch 20/20: 100%|██████████| 750/750 [00:58<00:00, 12.81it/s]


Epoch 20/20 | Train: 97.97% | Val: 91.92% | Time: 64.5s
✅ Training Complete! Best Val Acc: 92.08%


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
epoch_time,▆▄▄█▇▃▄▄▆▄▁▃▂▄▅▁▃▄▄▄
train_accuracy,▁▄▄▅▅▅▆▆▆▇▇▇▇▇▇█████
train_loss,█▅▄▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁
train_val_gap,▁▄▄▅▆▅▆▆▆▆▆▆▇▇██▇███
val_accuracy,▁▂▅▅▄▆▆▇▇███▇▇▆▇████
val_loss,█▆▄▃▄▃▃▂▂▁▁▁▃▂▄▄▂▃▃▃
best_val_accuracy,92.075
epoch,20
epoch_time,64.49963
final_val_accuracy,91.91667


## 10. Results Summary

Alle Experimente abgeschlossen! Gehe zu https://wandb.ai um die Ergebnisse zu sehen!

In [13]:
print("="*80)
print("✅ ALLE EXPERIMENTE ABGESCHLOSSEN!")
print("="*80)
print("\n📊 Gehe zu https://wandb.ai/")
print("   Projekt: Paper_4")
print("\n🎯 Du kannst jetzt:")
print("   1. Alle Runs im Dashboard vergleichen")
print("   2. Plots für dein Paper exportieren")
print("   3. Die Analyse schreiben")
print("="*80)

✅ ALLE EXPERIMENTE ABGESCHLOSSEN!

📊 Gehe zu https://wandb.ai/
   Projekt: Paper_4

🎯 Du kannst jetzt:
   1. Alle Runs im Dashboard vergleichen
   2. Plots für dein Paper exportieren
   3. Die Analyse schreiben
